In [0]:
%sql
CREATE OR REPLACE TABLE temporal_dimension (
  year INT,
  month INT,
  day INT,
  quarter INT,
  week_of_year INT,
  day_of_week INT,
  day_name STRING,
  month_name STRING,
  iso_date STRING,
  is_leap_year BOOLEAN,
  is_weekend BOOLEAN
);

INSERT INTO temporal_dimension
SELECT
  y AS year,
  m AS month,
  d AS day,
  CEIL(m / 3.0) AS quarter,
  weekofyear(TO_DATE(LPAD(CASE WHEN y < 1 THEN 1 ELSE y END, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0'))) AS week_of_year,
  dayofweek(TO_DATE(LPAD(CASE WHEN y < 1 THEN 1 ELSE y END, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0'))) AS day_of_week,
  date_format(TO_DATE(LPAD(CASE WHEN y < 1 THEN 1 ELSE y END, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0')), 'EEEE') AS day_name,
  date_format(TO_DATE(LPAD(CASE WHEN y < 1 THEN 1 ELSE y END, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0')), 'MMMM') AS month_name,
  CASE 
    WHEN y < 1 THEN LPAD(ABS(y)+1, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0') || ' BC'
    ELSE LPAD(y, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0')
  END AS iso_date,
  CASE 
    WHEN (y > 0 AND ((y % 4 = 0 AND y % 100 != 0) OR (y % 400 = 0))) THEN TRUE
    ELSE FALSE
  END AS is_leap_year,
  CASE 
    WHEN dayofweek(TO_DATE(LPAD(CASE WHEN y < 1 THEN 1 ELSE y END, 4, '0') || '-' || LPAD(m,2,'0') || '-' || LPAD(d,2,'0'))) IN (1,7) THEN TRUE
    ELSE FALSE
  END AS is_weekend
FROM (
  SELECT
    y,
    m,
    d
  FROM
    (
      SELECT explode(sequence(-3999, 3000)) AS y
    ) years
    CROSS JOIN (
      SELECT explode(sequence(1, 12)) AS m
    ) months
    CROSS JOIN (
      SELECT explode(sequence(1, 31)) AS d
    ) days
  WHERE
    -- Filter out invalid dates
    (m IN (1,3,5,7,8,10,12) AND d <= 31)
    OR (m IN (4,6,9,11) AND d <= 30)
    OR (m = 2 AND d <= 
      CASE 
        WHEN (y > 0 AND ((y % 4 = 0 AND y % 100 != 0) OR (y % 400 = 0))) THEN 29
        ELSE 28
      END
    )
);